<img src="img/logo_demcompare.png" width="100" align="right">

# Demcompare: introduction and basic usage

**Imports**

In [ ]:
import pyproj # pyproj as first import is necessary

In [ ]:
from snippets.utils_notebook import *

In [ ]:
from IPython.display import HTML, display
import tabulate
import pprint

# 1. INTRODUCTION

## 1.1. What is demcompare ? 

* Demcompare is a python software that aims at comparing two DEMs together.
* It performs the coregistration based on the Nuth & Kääb universal coregistration method.
* Two steps are available in demcompare coregistration's step: reprojection and coregistration 
* It provides a wide variety of standard metrics which can be classified.

## 1.2. Glossary

**DEM (Digital Elevation Model)** : a 3D computer graphics representation of elevation data to represent terrain.

**Coregistration** :  this step looks for offsets differences and align DEMs together

# 2. INPUTS

Demcompare user defines a dictionary as a configuration.

In [ ]:
config = {}

## 2.1. Outputs configuration

User needs to save results in an outputs directory

In [ ]:
config["output_dir"] = "./outputs/"

## 2.2. Inputs configuration

The user sets demcompare with a json configuration file. Each DEM is introduced with a dictionary. All parameters are described in the next chunk.

* For the coregistration step, two DEMs are necessary
    * input_ref and input_sec
    * input_sec is to be coregistered on input_ref
    * The inputs can have different size and resolution. 
    * By default, demcompare considers that the reference DEM has better resolution. 

|           Name           |                  Description                  |  Type  | Default value | Required |
|:------------------------:|:---------------------------------------------:|:------:|:-------------:|:--------:|
|     _path_               |             Path of the input Ref             | string |      None     |    Yes   |
|     _zunit_              |          Z axes unit of the input Ref         | string |       m       |    No    |
| _geoid_georef_           | True if the georef of the input Ref is geoid  |  bool  |     False     |    No    |
|  _geoid_path_            |          Geoid path of the input Ref          | string |      None     |    No    |
|    _nodata_              |         No data value of the input Ref        |   int  |      None     |    No    |
|  _classification_layers_ |        Path to the classification layer       | string |      None     |    No    |

User defines a configuration for both DEMs

In [ ]:
config["input_ref"] = { 
        "path": "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
        "zunit": "m",
        "georef": "WGS84"
    }

In [ ]:
config["input_sec"] = {      
        "path": "data/grenoble/Copernicus_blurred_and_shifted.tif",
        "zunit": "m",
        "nodata": -32768,
    }

## 2.3. Coregistration configuration

### Coregistration introduction

During the optional coregistration step, demcompare performs the coregistration on two uncoregistered DEMs like the ones below

Superposition of two DEMs that need to be coregistered
<img src="img/doc_ref.gif" width="300" align="center">

The user sets demcompare with a json configuration file. Each pipeline's step is introduced with a dictionary. All coregistration parameters are described in the next chunk.

|             Name            |                          Description                          |  Type  |    Default Value   | Required |
|:---------------------------:|:-------------------------------------------------------------:|:------:|:------------------:|----------|
|        _method_name_        |               Planimetric coregistration method               | string | nuth_kaab_internal | No       |
|    _number_of_iterations_   |       Number of iterations of the coregistration method       |   int  |          6         | No       |
| _estimated_initial_shift_x_ |            Estimated initial x coregistration shift           |   int  |          0         | No       |
| _estimated_initial_shift_y_ |            Estimated initial y coregistration shift           |   int  |          0         | No       |
|      _sampling_source_      |                Sampling source for reprojection               | string |         sec        | No       |
| _save_optional_outputs_ | If save coregistration method outputs such as iteration plots | string |        False       | No       |

User defines coregistration configuration.

In [ ]:
config["coregistration"] = {
    "method_name": "nuth_kaab_internal",
    "number_of_iterations": 6,
    "estimated_initial_shift_x": 0,
    "estimated_initial_shift_y": 0
  }

## 2.4. Statistic configuration

### Statistics introduction

Demcompare can compute a wide variety of statistics on either an input DEM, or the difference between two input DEMs. The statistics module can consider different number of inputs:

* If one single DEM is specified in the configuration; in this case the input or default metrics will directly be computed on the input DEM.
* If two DEMs are specified in the configuration; demcompare will do the reprojection for both DEMs to have the same resolution and size, and the difference between both reprojected DEMs will be considered to compute the input or default metrics.

The metrics to be computed may be specified at different levels on the statistics configuration:

* Global level: those metrics will be computed for all classification layers
* Classification layer level: those metrics will be computed specifically for the given classification layer



| Statistics |  |  |  |  |
|---|---|---|---|---|
| **Name** | **Description** | **Type** | **Default Value** | **Required** |
| remove_outliers | Remove outliers during statistics computation | string | False | No |
| metrics | Metrics to be computed | List | List of default metrics | No |

| Classification layer |  |  |  |  |
|---|---|---|---|---|
| **Name** | **Description** | **Type** | **Default Value** | **Required** |
| type | Classification layer type | string | None | Yes |
| remove_outliers | Remove outliers during statistics computation for this particular classification layer | string | Value set for the whole stats | No |
| nodata | Classification layer no data value | float or int | -32768 | No |
| metrics | Classification layer metrics to be computed (if metrics have been specified for the whole stats, they will also be computed for this classification) | List | List of default metrics | No |

| **Classification layer type** | **Name** | **Description** | **Type** | **Default Value** | **Required** |
|---|---|---|---|---|---|
| **Segmentation** | classes | Segmentation classes | Dict | None | Yes |
| **Slope** | ranges | Slope ranges | List | [0, 5, 10, 25, 40] | No |
| **Fusion** | ref | Ref classification layers to fusion | List | None | No |
|  | sec | Sec classification layers to fusion | List | None | No |

We use default statistics configuration

In [ ]:
config["statistics"] = {}

## 2.5. Complete configuration

In [ ]:
pprint.pprint(config, sort_dicts=False)

# 3. Load inputs

You must load the DEM with the `load_dem` function

In [ ]:
from demcompare.dem_tools import load_dem

In [ ]:
input_ref = load_dem(
    path=config["input_ref"]["path"], 
    zunit=config["input_ref"]["zunit"]
)

input_sec = load_dem(
    path=config["input_sec"]["path"], 
    zunit=config["input_sec"]["zunit"], 
)

We can see that there are differences in terms of size and resolution. But there is also an offset between them.

In [ ]:
show(stack_dems(input_ref, input_sec, "Referenced DEM and second DEM"))

# 4. Compute demcompare coregistration

We call the coregistration class

In [ ]:
from demcompare.coregistration import Coregistration

We create coregistration object

In [ ]:
coregistration_ = Coregistration(config["coregistration"])

The coregistration is computed and results are stored in transformation.

In [ ]:
transformation = coregistration_.compute_coregistration(input_sec, input_ref)

### Get offsets results from coregistration step

Different transformation's attributes are printed

In [ ]:
print(transformation)

The offsets are applied to original second dem 

In [ ]:
coreg_sec = transformation.apply_transform(input_sec)

Here, you can visualize Reference DEM with the coregistered second DEM.

In [ ]:
show(stack_dems(input_ref, coreg_sec, "Referenced DEM and second DEM"))

# 5. Compute demcompare statistics

## 5.1. Prerequisites for statistics computation

Coregistration is computed on reprojected DEMs with same size and resolution. They are stored and we need them for computing the statistics.

In [ ]:
reproj_ref = coregistration_.reproj_ref
reproj_sec = coregistration_.reproj_sec

Statistics must be computed on the altitude difference's of reprojected DEM, which is computed with the function `compute_alti_diff_for_stats`

In [ ]:
from demcompare.dem_tools import compute_alti_diff_for_stats

In [ ]:
altitude_diff = compute_alti_diff_for_stats(reproj_ref, reproj_sec)

In [ ]:
show_dem(altitude_diff, 
         "Altitude difference on reprojected DEM")

## 5.2. Computing the statistics

Import StatsProcessing class

In [ ]:
from demcompare.stats_processing import StatsProcessing

Create object from `StatsProcessing` with configuration and computed altitudes differences. If the input dem is an altitude difference the input_diff parameter is set to True. 

In [ ]:
stats_processing_ = StatsProcessing(config['statistics'], altitude_diff, input_diff=True)

Calculate metrics requested in the configuration and store the result in a StatsDataset object

In [ ]:
stats_dataset = stats_processing_.compute_stats()

### Get default metrics results from statistics step

Here we show all the calculated metrics on the global classification layer

In [ ]:
stats_metrics = stats_dataset.get_classification_layer_metrics(classification_layer="global")

In [ ]:
list_metrics = [["Metric's name", "Measured metrics"]]
for metric in stats_metrics: 
    value = stats_dataset.get_classification_layer_metric(classification_layer="global", metric=metric)
    list_metrics.append([metric, value[0]])
    
display(HTML(tabulate.tabulate(list_metrics, tablefmt='html', headers="firstrow")))